### **Métodos de Optimización**
#### Prácticas computacionales de "Programación Lineal"

Instrucciones para los Ejercicios

1. **Trabajo en Grupo:**
   - Los ejercicios deben ser resueltos y entregados en grupo.
   - La cantidad de integrantes por grupo será definida el día de la actividad, así como la fecha límite para la entrega.

2. **Uso de Google Colab y Compartir:**
   - Este notebook debe ser copiado al GitHub o Google Drive de alguno de los integrantes del grupo.
   - El grupo será responsable de programar las soluciones, realizar las pruebas y enviar el trabajo final al profesor.

3. **Implementación de los Ejercicios:**
   - Cada ejercicio debe ser implementado de manera que cumpla con los objetivos específicos descritos en cada problema.
   - El código debe devolver claramente la información calculada de acuerdo a lo solicitado.

4. **Calidad del Código:**
   - El código debe ejecutarse sin errores.
   - Es obligatorio incluir **comentarios explicativos** para describir las ideas y conceptos implícitos en el código, facilitando la comprensión de su lógica.

5. **Envío del Trabajo:**
   - Una vez completado, el notebook debe ser enviado a través de Moodle.
   - En caso de dudas, pueden contactarme por correo electrónico a **marcelo.danesi@utec.edu.uy**.

6. **Orientaciones Adicionales:**
   - Asegúrense de que todas las celdas de código hayan sido ejecutadas antes de enviar.
   - Incluyan el nombre completo y correo electrónico de todos los integrantes al inicio del notebook.
   - Si utilizan referencias externas, menciónenlas de forma adecuada.

¡Buena suerte y aprovechen la práctica para consolidar los conceptos de métodos optimización!

#### **Programación Lineal y Método Simplex**



#### **1) Maximización con Simplex (visión computacional)**

Un problema estándar de **maximización lineal** busca
$$
\max\; z = c^\top x \quad \text{s.a.}\quad A x \le b,\;\; x \ge 0,
$$
donde $x\in\mathbb{R}^n$ son las variables de decisión, $A\in\mathbb{R}^{m\times n}$ y $b\in\mathbb{R}^m$. En la práctica, las librerías numéricas (como `scipy.optimize.linprog`) resuelven $\min\; c^\top x$. Para resolver una $\max$, transformamos a $\min(-c^\top x)$. El método HiGHS (por defecto en `linprog`) ejecuta una variante robusta (simplex revisado/interior point según el caso), entregando soluciones eficientes incluso para dimensiones medianas-grandes.

##### **Ejemplo: Mezcla de producción (2 variables)**

$$
\begin{aligned}
\max\; z &= 3x_1 + 2x_2 \\
\text{s.a.}\;& 2x_1 + x_2 \le 100 \quad (\text{horas de trabajo})\\
& x_1 + 2x_2 \le 80 \quad (\text{materia prima})\\
& x_1, x_2 \ge 0.
\end{aligned}
$$
**Objetivo**  
Formular el LP en matrices $(c,A_{ub},b_{ub})$, resolver en Python, imprimir $z^*$, $x^*$ y analizar qué restricciones quedan activas (con folga cero).

In [1]:
# --- Sección 1: Maximización con Simplex (SciPy/HiGHS) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve max c^T x convirtiéndolo a min (-c)^T x.
    Retorna el objeto res de linprog; imprime z* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)   # x >= 0 por defecto
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    z = float(c @ res.x)  # revertimos el signo para el valor máximo
    print("z* =", z)
    print("x* =", res.x)
    # Folgas de las restricciones <= (si existen)
    if A_ub is not None:
        slack = b_ub - A_ub @ res.x
        print("slack (<=) =", slack)
    return res

# Datos del ejemplo (mezcla de producción)
c   = [3, 2]
Aub = [[2, 1],
       [1, 2]]
bub = [100, 80]

res = solve_lp_max(c, A_ub=Aub, b_ub=bub)

z* = 160.0
x* = [40. 20.]
slack (<=) = [0. 0.]


##### **Tarea 1.**

Sustituir $z=3x_1+2x_2$ por $z=4x_1+x_2$; resolver y *explicar* qué vértice del politopo queda activo y por qué.

##### **Tarea 2.**


Agregar la restricción $x_1 \le 50$; resolver y comparar $z^*$ con el caso base, comentando el efecto.

##### **Tarea 3.**

 Generar un LP aleatorio grande ($n=50$, $m=100$) con $A\sim U[0,1]$, $b\sim U[20,40]$, $c\sim U[0,1]$; resolver, medir tiempo de cómputo y reportar $z^*$.

#### **2) Minimización (primal con $\ge$ vs. dual)**

Para $\min\; c^\top x$ con $A x \ge b,\, x\ge 0$, hay dos estrategias prácticas:  
1. transformar a $-A x \le -b$ y usar `linprog` directamente;
2. construir el **dual** (que es $\max$ con $\le$) y resolverlo con la misma rutina de maximización. La equivalencia $C_{\min}=P_{\max}$ (dualicidad fuerte, bajo condiciones regulares) permite verificar resultados y reconstruir la solución primal por *complementariedad*.

##### **Ejemplo: Conversión directa ($Ax \ge b \to -Ax \le -b$).**

$$
\begin{aligned}
\min\; C &= 2x + y + 2z \\
\text{s.a. }& x + y + z \ge 4,\\
& 2x + y + 3z \ge 6,\\
& x,y,z \ge 0.
\end{aligned}
$$

**Objetivo**   
Transformar las restricciones $\ge$ a $\le$ multiplicando por $-1$. Resolver en Python e interpretar $C^*$ y $x^*$.   
*Opcional:* Montar y resolver el dual para verificar $C_{\min}=P_{\max}$.

In [ ]:
# --- Sección 2: Minimización (Ax >= b -> -Ax <= -b) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_min(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    """
    Resuelve min c^T x en forma directa (<= , =).
    Retorna el objeto res de linprog; imprime C* y x*.
    """
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)
    res = linprog(c=c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("C* =", float(c @ res.x))
    print("x* =", res.x)
    return res

c   = [2, 1, 2]
Age = np.array([[1, 1, 1],
                [2, 1, 3]])
bge = np.array([4, 6])

# Convertimos Ax >= b a -Ax <= -b
Aub = -Age
bub = -bge

res = solve_lp_min(c, A_ub=Aub, b_ub=bub)

##### **Tarea 1**

Cambiar los RHS (*Right-Hand Side* o lado derecho) a $b=(5,7)$; resolver y comparar $C^*$ con el caso base.

##### **Tarea 2.**

Agregar la cota $x \le 3$; resolver e indicar qué restricciones quedan activas en el óptimo.

##### **Tarea 3: Resolver el dual explícito.**

Construir el dual:
$$
\begin{aligned}
\max\; P&=4u+6v \\
\text{s.a. }& u+2v\le 2,\\
& u+v\le 1,\\
& u+3v\le 2,\\
& u,v\ge 0,
\end{aligned}
$$

resolver con la función de $\max$ y verificar $C_{\min}=P_{\max}$.


#### **3) Variantes (Two-Phase, Big-M, Dual Simplex)**

*Two-Phase* introduce variables artificiales para construir una base factible inicial (Fase 1) y luego optimizar el problema original (Fase 2). *Big-M* penaliza las artificiales en la función objetivo (FO), pero puede ser numéricamente sensible. *Dual Simplex* es útil cuando la base inicial es primal-infactible y dual-factible. En `linprog` (HiGHS), estrategias equivalentes se aplican internamente de forma robusta.


##### **Ejemplo: Restricción $\ge$ y $=$.**

$$
\begin{aligned}
\max\; z &= 3x_1 + x_2\\
\text{s.a.}\;& x_1 + x_2 \ge 4,\\
& x_1 + 2x_2 = 6,\\
& x_1, x_2 \ge 0.
\end{aligned}
$$

**Objetivo**   
Modelar $\ge$ como $-\le$, la igualdad en $A_{eq}$, resolver con `linprog` y comentar qué restricciones quedan activas.

In [ ]:
# --- Sección 3: Variante (>= y =) ---

import numpy as np
from scipy.optimize import linprog

def solve_lp_max(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None,
                 bounds=None, method="highs"):
    c = np.array(c, dtype=float)
    if bounds is None:
        bounds = [(0, None)] * len(c)
    res = linprog(c=-c, A_ub=A_ub, b_ub=b_ub,
                  A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method=method)
    print("z* =", float(c @ res.x))
    print("x* =", res.x)
    return res

c    = [3, 1]
Aub  = -np.array([[1, 1]])   # >= -> -<=
bub  = -np.array([4])
Aeq  = np.array([[1, 2]])
beq  = np.array([6])

_ = solve_lp_max(c, A_ub=Aub, b_ub=bub, A_eq=Aeq, b_eq=beq)

z* = 18.0
x* = [6. 0.]


##### **Tarea 1.**

Cambiar la igualdad a $x_1+2x_2=5$; resolver y comentar factibilidad y cambio en el óptimo.

##### **Tarea 2.**

 Sustituir $\ge$ por $x_1+x_2\ge 5$ y comparar el vértice óptimo con el caso base.


##### **Tarea 3.**

Implementar un *mini Two-Phase* para un LP $2\times 2$ con una restricción $\ge$:
1. Fase 1 con $w = \sum$ artificiales, alcanzar $w^*=0$;
2. Fase 2 optimizar la FO original. Validar con `linprog`.

#### **4) Problema de transporte (estructura y código)**

En el **transporte** balanceado, dadas ofertas $(\text{oferta}_i)$ y demandas $(\text{demanda}_j)$ con $\sum_i \text{oferta}_i = \sum_j \text{demanda}_j$, y costos unitarios $c_{ij}$, se minimiza
$$
\min\; Z = \sum_{i=1}^m\sum_{j=1}^n c_{ij} x_{ij}
\quad \text{s.a.}\quad
\sum_{j=1}^n x_{ij} = \text{oferta}_i,\;
\sum_{i=1}^m x_{ij} = \text{demanda}_j,\;
x_{ij}\ge 0.
$$
Vectorizando por filas, construimos $A_{eq}$ con $m$ ecuaciones de oferta y $n$ de demanda.

##### **Ejemplo: Altavoces ($2\times3$)**

Costos (\$):
$$
\begin{array}{c|ccc}
 & A & B & C \\ \hline
\text{I} & 20 & 8 & 10 \\
\text{II} & 12 & 22 & 18
\end{array}
\qquad
\text{Oferta: }(350,550),\quad \text{Demanda: }(200,300,400).
$$
**Objetivo:** Construir $c, A_{eq}, b_{eq}$ de forma automática, resolver con $\min$, imprimir el plan $x$ como matriz y analizar qué arcos quedan positivos en el óptimo.

In [ ]:
# --- Sección 4: Transporte (balanceado: Suma de ofertas = Suma de demandas) ---

import numpy as np
from scipy.optimize import linprog

def build_transport_lp(cost, supply, demand):
    """
    Devuelve c (aplanado por filas), A_eq, b_eq, bounds (x_ij >= 0).
    """
    cost   = np.asarray(cost, dtype=float)
    supply = np.asarray(supply, dtype=float)
    demand = np.asarray(demand, dtype=float)
    m, n = cost.shape
    c = cost.flatten()
    # Oferta: m ecuaciones
    A_sup = np.zeros((m, m*n))
    for i in range(m):
        A_sup[i, i*n:(i+1)*n] = 1.0
    # Demanda: n ecuaciones
    A_dem = np.zeros((n, m*n))
    for j in range(n):
        A_dem[j, j::n] = 1.0
    A_eq = np.vstack([A_sup, A_dem])
    b_eq = np.concatenate([supply, demand])
    bounds = [(0, None)] * (m*n)
    return c, A_eq, b_eq, bounds

def solve_lp_min(c, A_eq=None, b_eq=None, bounds=None, method="highs"):
    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method=method)
    if not res.success:
        print("⚠️ linprog:", res.message)
    print("Costo mínimo =", float(res.fun))
    return res

cost   = np.array([[20, 8, 10],
                   [12,22,18]])
supply = np.array([350, 550])
demand = np.array([200, 300, 400])

c, Aeq, beq, bnds = build_transport_lp(cost, supply, demand)
res = solve_lp_min(c, A_eq=Aeq, b_eq=beq, bounds=bnds)
print("Plan (matriz):\n", res.x.reshape(cost.shape))

Costo mínimo = 11600.0
Plan (matriz):
 [[  0. 300.  50.]
 [200.   0. 350.]]


##### **Tarea 1.**

 Cambiar el costo (I→B) de 8 a 9; resolver y comparar costo total y plan con el caso base.

##### **Tarea 2: Caso no balanceado.**

Aumentar la demanda de C a 450. Agregar un *origen ficticio* con oferta 50 y costos 0; resolver y comentar el papel del origen ficticio.

##### **Tarea 3: Instancia aleatoria balanceada ($5\times 7$.**

#### **5) Problema de asignación (algoritmo Húngaro)**

En **asignación** $n\times n$ se busca $\min \sum c_{ij} x_{ij}$ s.a. cada agente hace exactamente una tarea y cada tarea recibe exactamente un agente (variables binarias). Su estructura permite resolverlo eficientemente mediante el *algoritmo Húngaro* (Kuhn-Munkres). Para maximizar beneficios, se convierte a minimización restando de una constante $M\ge \max B_{ij}$.

##### **Ejemplo: $3\times 3$.**

$$
\begin{array}{c|ccc}
 & A & B & C \\ \hline
\text{T1} & 14 & 5 & 8 \\
\text{T2} & 2 & 12 & 6 \\
\text{T3} & 7 & 8 & 3
\end{array}
$$
**Objetivo:**   
Aplicar `linear_sum_assignment` para obtener la asignación óptima y verificar el costo mínimo.

In [ ]:
# --- Sección 5: Asignación (algoritmo Húngaro) ---

import numpy as np
from scipy.optimize import linear_sum_assignment

cost = np.array([[14, 5, 8],
                 [ 2,12, 6],
                 [ 7, 8, 3]])

rows, cols = linear_sum_assignment(cost)
print("Asignación (i->j):", list(zip(rows, cols)))
print("Costo mínimo =", cost[rows, cols].sum())

Asignación (i->j): [(np.int64(0), np.int64(1)), (np.int64(1), np.int64(0)), (np.int64(2), np.int64(2))]
Costo mínimo = 10
